<a href="https://colab.research.google.com/github/Naveensadanandan/Personal_fitness_coach/blob/main/Fitness_coach_project_RAGpipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document processing or embedding creation

In [1]:
import os
import requests

github = "https://raw.githubusercontent.com/Naveensadanandan/Personal_fitness_coach/main/data/"

file_names = ["ncert_textbook_fitness.pdf","human-nutrition-text.pdf","JN - UpperLower Program.pdf"]

save_dir = "Data/"

if not os.path.exists(save_dir):
  print(f"file path doesnt exist, downloading")
  os.makedirs(save_dir, exist_ok=True)

  for file_name in file_names:

    base_url = github + file_name

    response = requests.get(base_url)

    if response.status_code == 200:
      with open(os.path.join(save_dir,file_name),"wb") as f:
        f.write(response.content)
      print(f"file has been saved as {file_name}")
    else:
      print(f"[INFO] failed to downlaod : status code {response.status_code}")
else:
  print(f"Directory already exists, skipping download")

file path doesnt exist, downloading
file has been saved as ncert_textbook_fitness.pdf
file has been saved as human-nutrition-text.pdf
file has been saved as JN - UpperLower Program.pdf


In [2]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 64.2 MB/s eta 0:00:00


In [3]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text:str) -> str:
  cleaned_text = text.replace("\n"," ").strip()
  return cleaned_text

def read_write_pdf(filepath: str) -> list[dict]:
  combined_list = []
  for file_name in os.listdir(filepath):
    filepath_for_pdf = os.path.join(filepath,file_name)
    if os.path.isfile(filepath_for_pdf):
      doc = fitz.open(filepath_for_pdf)

      for page_number , page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        combined_list.append({"source" : file_name,
                                "page_number": page_number,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_setence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4, # 1 token = ~4 characters
                                "text": text})
  return combined_list

In [4]:
pages_and_texts = read_write_pdf(filepath=save_dir)
pages_and_texts[:2]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[{'source': 'ncert_textbook_fitness.pdf',
  'page_number': 0,
  'page_char_count': 95,
  'page_word_count': 19,
  'page_setence_count_raw': 1,
  'page_token_count': 23.75,
  'text': 'Health and Physical  Education Textbook for Class IX pre pages.indd   1 18-11-2016   4:25:29 PM'},
 {'source': 'ncert_textbook_fitness.pdf',
  'page_number': 1,
  'page_char_count': 1889,
  'page_word_count': 327,
  'page_setence_count_raw': 9,
  'page_token_count': 472.25,
  'text': 'First Edition November 2016 Kartika 1938 PD  5T BS ©\tNational Council of Educational  Research and Training, 2016 ` 135.00 Printed on 80 GSM paper with NCERT  watermark Published at the Publication Division  by the Secretary, National Council of  Educational Research and Training,   Sri Aurobindo Marg, New Delhi 110 016  and ................................... ISBN 978-93-5007-819-8 OFFICES OF THE PUBLICATION   DIVISION, NCERT NCERT Campus Sri Aurobindo Marg New  Delhi 110 016\t Phone :  011-26562708 108, 100 Feet Road  Hosd

In [5]:
import random
random.sample(pages_and_texts, k = 5)

[{'source': 'ncert_textbook_fitness.pdf',
  'page_number': 83,
  'page_char_count': 2122,
  'page_word_count': 407,
  'page_setence_count_raw': 23,
  'page_token_count': 530.5,
  'text': '72\t Health and Physical Education\u2009—\u2009Class IX • \tBat: The bat is also known as a paddle, or racket. The  blade shall be flat and made up of wood of dark matte  colour, one side black and other side bright red. But  majority of rackets are of similar size about 15cm across  and 25cm long including the handle. (see fig. 6.20) • \tScoreboard: The board, used for scoring is called Table  Tennis Scoreboard. •  Net Gauge is used to measure the height of the net and  also to measure the thickness of the racket and tension  of the bat rubber which should be as per International  Table Tennis Federation regulations. Dimensions Table Tennis Table\t Net Ball Shape Length N.A. Width\t N.A. Height\t\t N.A. Weight N.A. N.A. Diameter N.A. N.A. 6.6.3\t Events • \tSingles (Boys, Girls),  • \tDoubles (Boys, 

In [6]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df

,source,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,text
0,ncert_textbook_fitness.pdf,0,95,19,1,23.75,Health and Physical Education Textbook for Cl...
1,ncert_textbook_fitness.pdf,1,1889,327,9,472.25,First Edition November 2016 Kartika 1938 PD 5...
2,ncert_textbook_fitness.pdf,2,2187,364,17,546.75,"Foreword Many factors like, physical, psycholo..."
3,ncert_textbook_fitness.pdf,3,373,62,2,93.25,As an organisation committed to systemic refor...
4,ncert_textbook_fitness.pdf,4,902,129,8,225.50,Textbook Development Committee Members Bharati...
...,...,...,...,...,...,...,...
1506,JN - UpperLower Program.pdf,82,1049,168,23,262.25,JEFF NIPPARD’S - UPPER/LOWER SIZE AND STRENGTH...
1507,JN - UpperLower Program.pdf,83,2066,319,40,516.50,JEFF NIPPARD’S - UPPER/LOWER SIZE AND STRENGTH...
1508,JN - UpperLower Program.pdf,84,2085,313,44,521.25,JEFF NIPPARD’S - UPPER/LOWER SIZE AND STRENGTH...
1509,JN - UpperLower Program.pdf,85,854,131,16,213.50,JEFF NIPPARD’S - UPPER/LOWER SIZE AND STRENGTH...


In [7]:
file_names = ["ncert_textbook_fitness.pdf","human-nutrition-text.pdf","JN - UpperLower Program.pdf"]
#since every document page number doesnt start with 1 we need to change page number for evaluation in the future.
page_start = {"ncert_textbook_fitness.pdf" : 12,
              "human-nutrition-text.pdf" : 41,
              "JN - UpperLower Program.pdf" : 0}
for key, value in page_start.items():
  print(key,value)

ncert_textbook_fitness.pdf 12
human-nutrition-text.pdf 41
JN - UpperLower Program.pdf 0


In [8]:
df_copy = df.copy()
for key, value in page_start.items():
  df_copy.loc[df_copy["source"] == key, "page_number"] = df_copy["page_number"] - value
df_copy

,source,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,text
0,ncert_textbook_fitness.pdf,-12,95,19,1,23.75,Health and Physical Education Textbook for Cl...
1,ncert_textbook_fitness.pdf,-11,1889,327,9,472.25,First Edition November 2016 Kartika 1938 PD 5...
2,ncert_textbook_fitness.pdf,-10,2187,364,17,546.75,"Foreword Many factors like, physical, psycholo..."
3,ncert_textbook_fitness.pdf,-9,373,62,2,93.25,As an organisation committed to systemic refor...
4,ncert_textbook_fitness.pdf,-8,902,129,8,225.50,Textbook Development Committee Members Bharati...
...,...,...,...,...,...,...,...
1506,JN - UpperLower Program.pdf,82,1049,168,23,262.25,JEFF NIPPARD’S - UPPER/LOWER SIZE AND STRENGTH...
1507,JN - UpperLower Program.pdf,83,2066,319,40,516.50,JEFF NIPPARD’S - UPPER/LOWER SIZE AND STRENGTH...
1508,JN - UpperLower Program.pdf,84,2085,313,44,521.25,JEFF NIPPARD’S - UPPER/LOWER SIZE AND STRENGTH...
1509,JN - UpperLower Program.pdf,85,854,131,16,213.50,JEFF NIPPARD’S - UPPER/LOWER SIZE AND STRENGTH...


In [9]:
df_copy.describe()

,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count
count,1511.000000,1511.000000,1511.000000,1511.000000,1511.000000
mean,465.829914,1269.018531,221.774322,11.028458,317.254633
std,367.792908,654.278173,116.622741,7.283101,163.569543
min,-41.000000,0.000000,1.000000,1.000000,0.000000
25%,118.500000,818.500000,141.000000,5.000000,204.625000
50%,411.000000,1327.000000,229.000000,11.000000,331.750000
75%,788.500000,1701.500000,293.500000,16.000000,425.375000
max,1166.000000,3452.000000,627.000000,47.000000,863.000000


# Using Spacy to sentensize our text

In [10]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")

In [11]:
pages_and_texts = df_copy.to_dict(orient = "records")

In [12]:
for item in tqdm(pages_and_texts):
  item["sentences"] = list(nlp(item["text"]).sents)

  item["sentences"] = [str(sentense) for sentense in item["sentences"]]

  item["spacy_sentence_count"] = len(item["sentences"])

  0%|          | 0/1511 [00:00<?, ?it/s]

In [13]:
random.sample(pages_and_texts, k=1)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 415,
  'page_char_count': 1099,
  'page_word_count': 192,
  'page_setence_count_raw': 9,
  'page_token_count': 274.75,
  'text': 'Protein-rich  Foods by  Smastronard o / CC  BY-SA 4.0  Examples of complete protein sources include soy, dairy products,  meat, and seafood. Examples of incomplete protein sources include  legumes and corn.  Most plant-based foods are deficient in at least one essential  amino acid and therefore are incomplete protein sources. For  example, grains are usually deficient in the amino acid lysine, and  legumes are deficient in methionine or tryptophan. Because grains  and legumes are not deficient in the same amino acids they can  complement each other in a diet. Incomplete protein foods are  called complementary foods because when consumed in tandem  they contain all nine essential amino acids at adequate levels. Some  examples of complementary protein foods are given in Table 6.4  “Complementing Protein

In [14]:
from os.path import split
#chunking sentence togother
split_size_for_chunking = 10
def split_list(input_list : list,
               split_size : int = split_size_for_chunking) -> list[list]:
               chunk_list = [input_list[i:i+split_size] for i in range(0,len(input_list),split_size)]
               return chunk_list

test_list = list(range(25))

split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [15]:
for item in tqdm(pages_and_texts):
  item["sentence_chunks"] = split_list(input_list = item["sentences"], split_size= split_size_for_chunking)
  item["chunk_count"] = len(item["sentence_chunks"])

  0%|          | 0/1511 [00:00<?, ?it/s]

In [16]:
random.sample(pages_and_texts, k=1)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 889,
  'page_char_count': 1007,
  'page_word_count': 173,
  'page_setence_count_raw': 11,
  'page_token_count': 251.75,
  'text': 'Image by  Riley  McCullough  on  unsplash.co m / CC0  Late Adolescence  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  After puberty, the rate of physical growth slows down. Girls stop  growing taller around age sixteen, while boys continue to grow  taller until ages eighteen to twenty. One of the psychological and  emotional changes that takes place during this life stage includes  the desire for independence as adolescents develop individual  identities apart from their families.1 As teenagers make more and  more of their dietary decisions, parents or other caregivers and  authority figures should guide them toward appropriate, nutritious  choices. One way that teenagers assert their independence is by  choosing what to eat. They have their own 

## Creating an attribute dictionary for each chunks

In [17]:
import re

chunks = []
for item in tqdm(pages_and_texts):
  for chunk in item["sentence_chunks"]:
    chunk_dict = {}
    chunk_dict["source"] = item["source"]
    chunk_dict["page_number"] = item["page_number"]

    joined_sentence = "".join(chunk).replace("  "," ").strip()
    joined_sentence = re.sub(r'\.([A-Z])', r'. \1', joined_sentence)

    chunk_dict["sentence_chunk"] = joined_sentence

    # Get some stats on our chunks
    chunk_dict["chunk_char_count"] = len(joined_sentence)
    chunk_dict["chunk_word_count"] = len([word for word in joined_sentence.split(" ")])
    chunk_dict["chunk_token_count"] = len(joined_sentence) / 4 # 1 token = ~4 chars

    chunks.append(chunk_dict)

  0%|          | 0/1511 [00:00<?, ?it/s]

In [18]:
random.sample(chunks, k=1)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 993,
  'sentence_chunk': 'The Major Types of Foodborne Illness | 993',
  'chunk_char_count': 42,
  'chunk_word_count': 8,
  'chunk_token_count': 10.5}]

In [36]:
chunks[20]["source"]

'ncert_textbook_fitness.pdf'

In [19]:
df = pd.DataFrame(chunks)
df.head()

,source,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,ncert_textbook_fitness.pdf,-12,Health and Physical Education Textbook for Cla...,92,16,23.00
1,ncert_textbook_fitness.pdf,-11,First Edition November 2016 Kartika 1938 PD 5T...,1858,296,464.50
2,ncert_textbook_fitness.pdf,-10,"Foreword Many factors like, physical, psycholo...",1543,235,385.75
3,ncert_textbook_fitness.pdf,-10,Experiential learning activities for acquiring...,605,91,151.25
4,ncert_textbook_fitness.pdf,-9,As an organisation committed to systemic refor...,367,56,91.75


In [20]:
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,2522.00,2522.00,2522.00,2522.00
mean,450.75,742.84,115.85,185.71
std,370.34,448.76,72.68,112.19
min,-41.00,12.00,3.00,3.00
25%,115.00,347.00,52.00,86.75
50%,354.50,739.00,115.00,184.75
75%,788.75,1097.00,172.00,274.25
max,1166.00,2353.00,357.00,588.25


## Deleting chunks whaving less than minimum token count

In [21]:
min_token_count = 30
pages_and_chunks_mintoken = df[df["chunk_token_count"] > min_token_count].to_dict( orient = "records")
len(pages_and_chunks_mintoken)

2325


## Check the link for details about pre trained models
[Semantic Search Models](https://sbert.net/docs/sentence_transformer/pretrained_models.html#semantic-search-models)

In [22]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 8.8 MB/s eta 0:00:00


In [23]:
from sentence_transformers import SentenceTransformer
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

embedding_model = SentenceTransformer("multi-qa-mpnet-base-cos-v1", device = device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
#saving encoding

%%time

for item in tqdm(pages_and_chunks_mintoken):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/2325 [00:00<?, ?it/s]

CPU times: user 46.4 s, sys: 612 ms, total: 47.1 s
Wall time: 54.5 s


In [25]:
# lets embedd our data

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_mintoken]

In [26]:
text_chunks_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=16, # you can experiment to find which batch size leads to best results
                                               convert_to_tensor=True)
text_chunks_embeddings

tensor([[ 0.0609, -0.0351, -0.0092,  ..., -0.0087, -0.0518, -0.0260],
        [-0.0080, -0.0325, -0.0305,  ...,  0.0041, -0.0058, -0.0262],
        [ 0.0493, -0.0224, -0.0399,  ..., -0.0058,  0.0140, -0.0549],
        ...,
        [-0.0289,  0.0151, -0.0078,  ..., -0.0167, -0.0236, -0.0304],
        [ 0.0289, -0.0084, -0.0163,  ...,  0.0199, -0.0548, -0.0404],
        [-0.0167,  0.0667, -0.0127,  ..., -0.0337, -0.0300, -0.0181]],
       device='cuda:0')

In [27]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_mintoken)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [28]:
text_chunks_embeddings.shape

torch.Size([2325, 768])

# Write a query and use semantic search for finding related topics

In [41]:
from sentence_transformers import SentenceTransformer,util
import torch
from time import perf_counter as timer


device = "cuda" if torch.cuda.is_available() else "cpu"

embedding_model = SentenceTransformer("multi-qa-mpnet-base-cos-v1", device = device)

query = "which workout is best for upperbody"

print(f"query : {query}")

query_embedding = embedding_model.encode(query,convert_to_tensor=True, device = device)
start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b = text_chunks_embeddings)[0]
end_time = timer()

print(f"[INFO] Time taken to get scores on {len(text_chunks_embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results (we'll keep top 5)
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


query : which workout is best for upperbody
[INFO] Time taken to get scores on 2325 embeddings: 0.00017 seconds.


torch.return_types.topk(
values=tensor([0.6070, 0.6066, 0.6035, 0.6021, 0.6020], device='cuda:0'),
indices=tensor([2278, 2276, 2280, 2277, 2279], device='cuda:0'))

In [33]:
text_chunks[2276]

'UPPER #2 SETS REPS RPE/%1RM REST 1 2 3 4 NOTES LSRPE BARBELL OVERHEAD PRESS 2 4 80% 2-3MIN SQUEEZE YOUR GLUTES TO KEEP YOUR TORSO UPRIGHT WIDE-GRIP PULL-UP 3 6 RPE7 2-3min PULL WITH YOUR CHEST TO THE BAR BARBELL CLOSE GRIP BENCH PRESS 3 10 65% 2-3min SHOULDER WIDTH GRIP BARBELL BENT OVER ROW 3 10 RPE8 2-3min PULL TO YOUR UPPER ABS PUSH-UP 2 AMRAP/2 RPE8 1-2min SQUEEZE YOUR PECS BAND PULL-APART 2 30 RPE8 1-2min PULL VIA SHOULDER EXTENSION ECCENTRIC-ACCENTUATED HAMMER CURL 3 10 RPE8 1-2min 3-SECOND LOWERING PHASE UPPER BODY WEAK POINT 1 2 or 3 15-20 RPE9 1-2min FOCUS ON MIND-MUSCLE CONNECTION LOWER #3 SETS REPS RPE/%1RM REST 1 2 3 4 NOTES LSRPE BACK SQUAT 3 8 75% 3-4MIN SIT BACK AND DOWN, 15° TOE FLARE, DRIVE YOUR KNEES OUT LATERALLY DUMBBELL WALKING LUNGE 3 15 RPE8 2-3min 15 STEPS PER LEG DUMBBELL STEP-UP 3 12 RPE9 0min SET THE BOX TO ~PARALLEL REVERSE HYPER 3 15 RPE9 1-2min FOCUS ON SQUEEZING YOUR GLUTES CABLE STANDING HIP ABDUCTION 3 10 RPE8 1-2min SQUEEZE YOUR GLUTES PLANK 3 :30 RPE

In [42]:
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    return wrapped_text

In [45]:
text_chunks[2]

'Experiential learning activities for acquiring skills for healthy living are made an integral part of the book. NCERT appreciates the hard work done by the Textbook Development Committee. I am highly thankful to Professor Saroj Yadav, Dean (Academic) and Project Coordinator of National Population Education Project (NPEP) and Adolescence Education Programme (AEP) for her continuous effort to develop this material in the present form. Several experts and teachers contributed towards the finalisation of this book. We are grateful to them and their institutions. pre pages.indd  3 18-11-2016  4:25:29 PM'

In [46]:
def print_top5(query:str, embedding_model = embedding_model):
  query_embedding = embedding_model.encode(query, convert_to_tensor=True, device =  device)
  dot_scores = util.dot_score(a=query_embedding, b =text_chunks_embeddings )[0]
  top_results = torch.topk(dot_scores, k =5)

  for score , idx in zip(top_results[0],top_results[1]):
    print(f"score : {score}")
    print(f"text")
    print(print_wrapped(text_chunks[idx]))
    # print_wrapped(pages_and_chunks_mintoken[idx])
    print(f"source : {pages_and_chunks_mintoken[idx]['source']}")
    print(f"page_number : {pages_and_chunks_mintoken[idx]['page_number']}")

In [47]:
print_top5(query = "how often do you need to workout")

score : 0.5935137271881104
text
Image by Allison Calabrese / CC BY 4.0 Physical Activity Recommendations The CDC
along with the American College of Sports Medicine (ACSM) have evidence based
 recommendations and guidelines for individuals to follow in order to obtain or
maintain a healthy lifestyle. Adults should get at least 150 minutes of
moderate- intensity aerobic physical activity or 75 minutes of vigorous-
intensity aerobic physical activity each week. In addition to aerobic physical
activity, it is recommended that adults do muscle strengthening activities on
each major muscle group two or three times each week. Adults also are
recommended by the ACSM to do flexibility exercises at least two to three times
a week to improve range of motion. To learn more about these guidelines visit
the CDC website at https://health.gov/paguidelines/guidelines/adults.aspx and
the ACSM website at http://www.acsm.org/about-acsm/ media-room/news-
releases/2011/08/01/acsm-issues-new- recommendations